# 以管線作業身分執行腳本

管線可讓您將多個步驟分組成一個工作流程。 您可以使用元件建置管線。 每個元件都會反映要執行的 Python 腳本。 元件定義于 YAML 檔案中，該檔案會指定腳本以及如何執行它。 

## 開始之前

您將需要最新版的  **azureml-ai-ml** 套件，才能在此筆記本中執行程式碼。 執行下列資料格以確認已安裝它。

> **注意**：
> 如果未安裝 **azure-ai-ml** 套件，請執行 `pip install azure-ai-ml` 以安裝它。

In [ ]:
## 連線到您的工作區

安裝必要的 SDK 套件之後，現在您已準備好連線到您的工作區。

若要連線到工作區，我們需要識別碼參數 - 訂用帳戶識別碼、資源組名和工作區名稱。 資源組名和工作區名稱已為您填入。 您只需要訂用帳戶識別碼才能完成命令。

若要尋找必要的參數，請按一下 Studio 右上方的訂用帳戶和工作區名稱。 窗格會在右側開啟。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 複製訂用帳戶識別碼，並將 **YOUR-SUBSCRIPTION-ID** 取代為您複製的值。 </p>

## 建立腳本

您將使用兩個步驟來建置管線：

1. **準備資料**：修正遺漏的資料，並將資料正規化。
1. **定型模型**：定型決策樹分類模型。

執行下列儲存格來建立 **src** 資料夾和兩個腳本。

In [ ]:
## 定義元件

若要定義您需要指定的元件：

- **中繼資料**： *名稱*、 *顯示名稱*、 *版本*、 *描述*、 *類型* 等。中繼資料有助於描述和管理元件。
- **介面**：*輸入和輸出**。* 例如，模型定型元件會將定型資料和正規化速率作為輸入，並產生定型的模型檔案作為輸出。 
- **命令、程式碼&環境**：*執行元件的命令*、程式*代碼**和環境。* 命令是執行元件的殼層命令。 程式碼通常是指原始程式碼目錄。 環境可以是 AzureML 環境， (策劃或自訂建立) 、docker 映射或 conda 環境。

執行下列資料格，為您想要以管線步驟執行的每個元件建立 YAML。

In [ ]:
## 載入元件

現在您已定義每個元件，您可以藉由參考 YAML 檔案來載入元件。 

## 建置管線

建立和載入元件之後，您可以建置管線。 您將將這兩個元件組成管線。 首先，您會想要讓 `prep_data` 元件執行。 第一個元件的輸出應該是第二個元件的 `train_decision_tree` 輸入，這會定型模型。

函式 `diabetes_classification` 代表完整的管線。 函式需要一個輸入變數： `pipeline_job_input` 。 資料資產是在安裝期間建立的。 您將使用已註冊的資料資產作為管線輸入。 

In [ ]:
您可以列印 `pipeline_job` 物件來擷取管線作業的組態：

In [ ]:
您可以藉由參考 參數並指定新值，來變更管線作業組態的任何參數：

In [ ]:
## 提交管線作業

最後，當您建置管線並設定管線作業視需要執行時，您可以提交管線作業：

## Define the components

To define the component you need to specify:

- **Metadata**: *name*, *display name*, *version*, *description*, *type* etc. The metadata helps to describe and manage the component.
- **Interface**: *inputs* and *outputs*. For example, a model training component will take training data and the regularization rate as input, and generate a trained model file as output. 
- **Command, code & environment**: the *command*, *code* and *environment* to run the component. Command is the shell command to execute the component. Code usually refers to a source code directory. Environment could be an AzureML environment (curated or custom created), docker image or conda environment.

Run the following cells to create a YAML for each component you want to run as a pipeline step.

In [ ]:
%%writefile prep-data.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_data
display_name: Prepare training data
version: 1
type: command
inputs:
  input_data: 
    type: uri_file
outputs:
  output_data:
    type: uri_file
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python prep-data.py 
  --input_data ${{inputs.input_data}}
  --output_data ${{outputs.output_data}}

In [ ]:
%%writefile train-model.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
display_name: Train a decision tree classifier model
version: 1
type: command
inputs:
  training_data: 
    type: uri_file
  reg_rate:
    type: number
    default: 0.01
outputs:
  model_output:
    type: mlflow_model
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python train-model.py 
  --training_data ${{inputs.training_data}} 
  --reg_rate ${{inputs.reg_rate}} 
  --model_output ${{outputs.model_output}} 

## Load the components

Now that you have defined each component, you can load the components by referring to the YAML files. 

In [ ]:
from azure.ai.ml import load_component
parent_dir = ""

prep_data = load_component(source=parent_dir + "./prep-data.yml")
train_decision_tree = load_component(source=parent_dir + "./train-model.yml")

## Build the pipeline

After creating and loading the components, you can build the pipeline. You'll compose the two components into a pipeline. First, you'll want the `prep_data` component to run. The output of the first component should be the input of the second component `train_decision_tree`, which will train the model.

The `diabetes_classification` function represents the complete pipeline. The function expects one input variable: `pipeline_job_input`. A data asset was created during setup. You'll use the registered data asset as the pipeline input. 

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification(pipeline_job_input):
    clean_data = prep_data(input_data=pipeline_job_input)
    train_model = train_decision_tree(training_data=clean_data.outputs.output_data)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
    }

pipeline_job = diabetes_classification(Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-data:1"))

You can retrieve the configuration of the pipeline job by printing the `pipeline_job` object:

In [ ]:
print(pipeline_job)

You can change any parameter of the pipeline job configuration by referring to the parameter and specifying the new value:

In [ ]:
# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"
# set pipeline level compute
pipeline_job.settings.default_compute = "aml-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# print the pipeline job again to review the changes
print(pipeline_job)

## Submit the pipeline job

Finally, when you've built the pipeline and configured the pipeline job to run as required, you can submit the pipeline job:

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes"
)
pipeline_job